## Jupyter Notebook Width Extension

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
order = 1

## Import Packages

In [ ]:
import numpy as np
import time
import tensorflow as tf
import pandas as pd
import random
import os
import gc
TF_ENABLE_DEPRECATION_WARNINGS=1

## GPU_configuration

In [ ]:
config = tf.compat.v1.ConfigProto(graph_options=tf.compat.v1.GraphOptions(optimizer_options=tf.compat.v1.OptimizerOptions(opt_level=tf.compat.v1.OptimizerOptions.L0)))
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
config.gpu_options.per_process_gpu_memory_fraction=0.1
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())

## Function Definition

In [ ]:
def sweep(x):
    div = x.sum(axis=1, dtype='float')
    res = x/div
    where_are_NaNs = np.isnan(res)
    res[where_are_NaNs] = 0
    return res

def scale(y, c=True, sc=True):
    x = y.copy()

    if c:
        x -= x.mean()
    if sc and c:
        x /= x.std()
    elif sc:
        x /= np.sqrt(x.pow(2).sum().div(x.count() - 1))
    return x

def shape(tensor):
    s = tensor.get_shape()
    return tuple([s[i].value for i in range(0, len(s))])

## Load Dataset

In [ ]:
# st_gsea_all_go_names = ["T5259"]
st_gsea_all_go_names = ["T4235", "T3799", "T5498", "T5359", "T3870", "T4857", "T5259"]
# st_gsea_all_go_names = ["T0081", "T4424", "T4515", "T4839", "T2791","T2465", "T5385"]
# l = "01"

## Main

In [ ]:
# for l in ["05","07",'09','11','13','15']:
for l in ["01"]:
    print("======= START =======")
    for a in list(range(len(st_gsea_all_go_names))):
        df = pd.read_csv('csv/' + st_gsea_all_go_names[a] + '.csv')
#         adj = np.asmatrix(pd.read_csv('csv/adj_' + l + '/' + st_gsea_all_go_names[a] + '.csv'))
        adj = np.asmatrix((pd.read_csv('csv/adj_' + l + '/' + st_gsea_all_go_names[a] + '.csv')).drop(columns="Unnamed: 0").values)
    #             print("Sample name is: " + st_gsea_all_go.names[a])
    #             print("Adjacency matrix: " + order_01_trimmed_adj_matrices_list.names[b])
        pathways_name = list(np.concatenate(df[["Unnamed: 0"]].values.tolist()))
        pathways = np.asmatrix(df.drop(columns="Unnamed: 0").values)
        pl = len(pathways)
        row = list(range(pl))
        p_val = []
        n=1
        para = [row[i * n:(i + 1) * n] for i in range((len(row) + n - 1) // n )]
        for i in para[order]:
            try:
                df_tmp = pd.read_csv(str(l) + '/' + st_gsea_all_go_names[a] + '_' + str(l) + '/' + st_gsea_all_go_names[a] + '_' + str(i) + '.csv')
                if len(df_tmp) == 16:
                    pass
                else:
                    for j in list(range(pl)):
                        pathway_i = pathways_name[i]
                        pathway_j = pathways_name[j]
                        combo_name = pathway_i + '_x_' + pathway_j
                        if combo_name in  df_tmp['combo_name'].tolist():
                            continue
                        else:
                            config = tf.ConfigProto(graph_options=tf.GraphOptions(optimizer_options=tf.OptimizerOptions(opt_level=tf.OptimizerOptions.L0)))
                            config.gpu_options.visible_device_list = "0"
                            sess = tf.Session(config=config)
                            sess.run(tf.global_variables_initializer())
                            obj1 = tf.constant(scale(pathways[i,]).getT(),dtype=tf.float32)
                            obj2 = tf.constant(scale(pathways[j,]),dtype=tf.float32)
                            prod = tf.matmul(obj1, obj2)
                            adj2 = tf.constant(sweep(adj),dtype=tf.float32)
                            local_scc = tf.einsum('ij,ij->i', adj2,prod)
                            local_scc_ = sess.run(local_scc)
                            local_scc_list = list(local_scc_)
                            global_scc = np.float32(np.mean(local_scc_list))
                            compare = []
                            num_row = list(range(max(obj1.shape)))
                            del prod
                            del local_scc
                            gc.collect()
                            sess.close()
                #                     print("================================================ Permutation Test ===================================================================")
                            start = time.time()
                            for k in list(range(100)):
                                sess = tf.compat.v1.Session(config=config)
                                rng = np.random.default_rng(seed = k)
                                new_order = rng.permutation(num_row,0)
                                x = tf.gather(obj1,new_order)
                                y = tf.gather(tf.transpose(obj2),new_order)
                                y = tf.transpose(y)
                                prod_shuff = tf.tensordot(x, y, axes=1)
                                final_shuff = tf.einsum('ij,ij->i', adj2, prod_shuff)
                                scc_shuff = tf.reduce_mean(final_shuff,0)
                                scc_shuff_ = sess.run(scc_shuff)
                                scc_shuff_ = scc_shuff_
                                compare.append(scc_shuff_)
                                sess.close()
                                del rng
                                del new_order
                                del scc_shuff
                                del scc_shuff_
                                del final_shuff
                                del prod_shuff
                                del x
                                del y
                                gc.collect()
                            del obj1
                            del obj2
                            compare = list(compare)
                            pval1=0
                            pval2=0
                            if global_scc < 0:
                                for p in compare:
                                    if p < global_scc:
                                        pval1 = pval1 + 1
                            else:
                                for p in compare:
                                    if p >= global_scc:
                                        pval2 = pval2 + 1

                            p_val = (pval1 + pval2)/100
                            gc.collect()
                            tf.keras.backend.clear_session()

                            path = str(l) + '/' + st_gsea_all_go_names[a] + '_' + str(l)
                            isExist = os.path.exists(path)
                            if not isExist:
                                os.makedirs(path)
                            if j == 0:
                                df2 = 0
                                df2 = pd.DataFrame({"combo_name":[combo_name],'local_scc':[local_scc_list], 'global_scc':[global_scc], 'permutation':[compare],'p_val':[p_val]})
                                df2.to_csv(path + '/' + st_gsea_all_go_names[a] + "_" + str(i) + '.csv',index = None)
                            else:
                                df_tmp.loc[len(df_tmp.index)] = [combo_name, local_scc_list, global_scc, compare, p_val]
                                df_tmp.to_csv(path + '/' + st_gsea_all_go_names[a] + "_" + str(i) + '.csv',index = None)
                            end = time.time()
                            elapsed = (end - start)
                            print(elapsed)
            except:

                for j in list(range(pl)):

                    pathway_i = pathways_name[i]
                    pathway_j = pathways_name[j]
                    combo_name = pathway_i + '_x_' + pathway_j
        #                     print("====================================================================================================================================")
        #                     print("= Sample name: " + st_gsea_all_go_names[a])
        #                     print("= Adjacency matrix: " + order_01_trimmed_adj_matrices_list.names[b])
        #                     print("= Combo name: [ %s ]"%(combo_name))
        #                     print("====================================================================================================================================")
                    config = tf.ConfigProto(graph_options=tf.GraphOptions(optimizer_options=tf.OptimizerOptions(opt_level=tf.OptimizerOptions.L0)))
                    config.gpu_options.visible_device_list = "0"
                    sess = tf.Session(config=config)
                    sess.run(tf.global_variables_initializer())
                    obj1 = tf.constant(scale(pathways[i,]).getT(),dtype=tf.float32)
                    obj2 = tf.constant(scale(pathways[j,]),dtype=tf.float32)
                    prod = tf.matmul(obj1, obj2)
                    adj2 = tf.constant(sweep(adj),dtype=tf.float32)
                    local_scc = tf.einsum('ij,ij->i', adj2,prod)
                    local_scc_ = sess.run(local_scc)
                    local_scc_list = list(local_scc_)
                    global_scc = np.float32(np.mean(local_scc_list))
                    compare = []
                    num_row = list(range(max(obj1.shape)))
                    del prod
                    del local_scc
                    gc.collect()
                    sess.close()
        #                     print("================================================ Permutation Test ===================================================================")
                    start = time.time()
                    for k in list(range(100)):
                        sess = tf.compat.v1.Session(config=config)
                        rng = np.random.default_rng(seed = k)
                        new_order = rng.permutation(num_row,0)
                        x = tf.gather(obj1,new_order)
                        y = tf.gather(tf.transpose(obj2),new_order)
                        y = tf.transpose(y)
                        prod_shuff = tf.tensordot(x, y, axes=1)
                        final_shuff = tf.einsum('ij,ij->i', adj2, prod_shuff)
                        scc_shuff = tf.reduce_mean(final_shuff,0)
                        scc_shuff_ = sess.run(scc_shuff)
                        scc_shuff_ = scc_shuff_
                        compare.append(scc_shuff_)
                        sess.close()
                        del rng
                        del new_order
                        del scc_shuff
                        del scc_shuff_
                        del final_shuff
                        del prod_shuff
                        del x
                        del y
                        gc.collect()

                    del obj1
                    del obj2
                    compare = list(compare)
                    pval1=0
                    pval2=0
                    if global_scc < 0:
                        for p in compare:
                            if p < global_scc:
                                pval1 = pval1 + 1
                    else:
                        for p in compare:
                            if p >= global_scc:
                                pval2 = pval2 + 1

                    p_val = (pval1 + pval2)/100

                    gc.collect()
                    tf.keras.backend.clear_session()

                    path = str(l) + '/' + st_gsea_all_go_names[a] + '_' + str(l)
                    isExist = os.path.exists(path)
                    if not isExist:
                        os.makedirs(path)
                    if j == 0:
                        df2 = 0
                        df2 = pd.DataFrame({"combo_name":[combo_name],'local_scc':[local_scc_list], 'global_scc':[global_scc], 'permutation':[compare],'p_val':[p_val]})
                        df2.to_csv(path + '/' + st_gsea_all_go_names[a] + "_" + str(i) + '.csv',index = None)
                    else:
                        df2.loc[len(df2.index)] = [combo_name, local_scc_list, global_scc, compare, p_val]
                        df2.to_csv(path + '/' + st_gsea_all_go_names[a] + "_" + str(i) + '.csv',index = None)
                        
                    end = time.time()
                    elapsed = (end - start)
                    print(elapsed)